In [8]:
#左键抠图，右键完成抠图并输出到output文件夹
import cv2
import numpy as np
input_img = cv2.imread('./resource/cat.jpg')

b,g,r = cv2.split(input_img)
alpha = np.ones(b.shape,dtype = b.dtype)*255#设置透明通道

height, width= input_img.shape[0:2]
tmp_img = input_img.copy()
mask = np.zeros_like(input_img)#初始化遮罩

point = []

def lasso(event, x, y, flags, param):
    global point,mask
    if event == cv2.EVENT_LBUTTONDOWN:
        point.append((x, y))
        print(point[-1])
        #连接各个节点
        if len(point) > 1:
            cv2.line(input_img, point[-2], point[-1], (0, 255, 0), 2)
            cv2.line(mask, point[-2], point[-1], (255,255, 255), 2)
        
        cv2.imshow('select', input_img)
    
    elif event == cv2.EVENT_RBUTTONDOWN:
        #组成闭合多边形，并保存掩码内的图片，其它所有像素设置为透明
        if len(point) > 2:
            cv2.line(input_img, point[-1], point[0], (0, 255, 0), 2)
            cv2.line(mask, point[-1], point[0], (255, 255, 255), 2)
            
            #填充mask多边形
            cv2.fillPoly(mask, [np.array(point)], (255, 255, 255))
            #扣出遮罩内容
            output_img = cv2.bitwise_and(tmp_img, mask)
            
            mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)  
            alpha[mask_gray==0] = 0
            
            #合并透明通道
            output_img = cv2.merge((output_img, alpha))
            cv2.namedWindow('output_img', cv2.WINDOW_NORMAL)   
            cv2.imshow('output_img', output_img)
            cv2.imwrite('./output/selected_area.png', output_img)
            
        
cv2.namedWindow('select', cv2.WINDOW_NORMAL)        
cv2.setMouseCallback('select', lasso)

cv2.imshow('select', input_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


(1267, 456)
(1257, 410)
(1255, 365)
(1273, 360)
(1292, 363)
(1311, 379)
(1344, 391)
(1363, 382)
(1400, 356)
(1436, 349)
(1475, 348)
(1505, 348)
(1525, 348)
(1553, 330)
(1604, 300)
(1615, 300)
(1629, 329)
(1631, 355)
(1637, 403)
(1644, 422)
(1679, 472)
(1691, 511)
(1691, 542)
(1684, 596)
(1673, 635)
(1667, 658)
(1673, 683)
(1687, 713)
(1692, 744)
(1695, 787)
(1680, 816)
(1659, 861)
(1647, 888)
(1623, 943)
(1620, 976)
(1621, 1035)
(1635, 1105)
(1651, 1141)
(1676, 1146)
(1699, 1176)
(1707, 1224)
(1696, 1251)
(1667, 1258)
(1604, 1243)
(1569, 1239)
(1549, 1203)
(1535, 1167)
(1516, 1119)
(1495, 1084)
(1463, 1059)
(1440, 1055)
(1453, 1109)
(1476, 1145)
(1499, 1189)
(1520, 1214)
(1535, 1260)
(1521, 1263)
(1485, 1265)
(1441, 1267)
(1392, 1269)
(1369, 1267)
(1352, 1238)
(1347, 1214)
(1332, 1219)
(1319, 1241)
(1304, 1250)
(1279, 1251)
(1212, 1250)
(1139, 1248)
(1105, 1248)
(1059, 1246)
(1027, 1246)
(981, 1239)
(957, 1224)
(921, 1205)
(877, 1157)
(859, 1134)
(828, 1110)
(779, 1098)
(684, 1098)
(60

In [11]:
#左键鼠标，可以跟着图像会移动到鼠标位置，按住左键移动，拖动图像

import cv2
import numpy as np

ori_img = cv2.imread('./resource/grass.jpg')
#将原图片转换成四通道图像
alpha1 = 255
alpha_channel = np.full((ori_img.shape[0], ori_img.shape[1]), alpha1, dtype=np.uint8)
ori_img_with_alpha = cv2.merge((ori_img, alpha_channel))
insert_img = cv2.imread('./output/selected_area.png',cv2.IMREAD_UNCHANGED) # 读取时保留 Alpha 通道
output_img = ori_img_with_alpha.copy()
#cv2.imshow('tmp',insert_img)

dragging = False 
size = 1

print(ori_img.shape)
print(insert_img.shape)

#对滑动条的数字进行映射[0,5]范围内
def on_trackbar(val):
    global size
    size = val/10000.0

# 计算非透明部分的边界
def calculate_non_transparent_bounds(image):
    alpha_channel = image[:, :, 3]
    coords = np.column_stack(np.where(alpha_channel > 0))
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    return (x_min, y_min, x_max, y_max)

#图像嵌入函数
def insert(x,y):
    global size
    #print(size)
    tmp_img = ori_img_with_alpha.copy() 
    # 计算插入图像的宽高
    resize_insert_img = cv2.resize(insert_img,(int(insert_img.shape[1] * size), int(insert_img.shape[0] * size)))
    h, w = resize_insert_img.shape[:2]
    
     # 计算非透明部分的边界
    x_min, y_min, x_max, y_max = calculate_non_transparent_bounds(resize_insert_img)
    
    # 计算插入位置，使非透明部分的边缘对齐鼠标
    insert_x = x - x_min
    insert_y = y - y_min
    
    #计算有效区域（背景图像内）
    y1 = max(insert_y, 0)
    y2 = min(insert_y + h, tmp_img.shape[0])
    x1 = max(insert_x, 0)
    x2 = min(insert_x + w, tmp_img.shape[1])
    
    # 计算插入图像的有效部分
    insert_y1 = y1-insert_y
    insert_y2 = insert_y1 + (y2 - y1)
    insert_x1 = x1-insert_x
    insert_x2 = insert_x1 + (x2 - x1)
    
    region = tmp_img[y1:y2, x1:x2]
    
    #处理透明图像
    alpha_s = resize_insert_img[insert_y1:insert_y2, insert_x1:insert_x2,3]/255.0
    alpha_l = 1.0 - alpha_s
        
    for c in range(0,3):
        tmp_img[y1:y2, x1:x2, c] = (alpha_s * resize_insert_img[insert_y1:insert_y2, insert_x1:insert_x2, c] +alpha_l * region[:, :, c])
    
    #嵌入图像
    return tmp_img

def process_img(event, x, y, flags, param):
    global dragging , Flag,output_img
    if event == cv2.EVENT_LBUTTONDOWN:
        Flag = True
        dragging = True
        output_img = insert(x,y)
        cv2.imshow('insert_img', output_img)
    elif event == cv2.EVENT_MOUSEMOVE:
        if dragging:
            output_img = insert(x,y)
            cv2.imshow('insert_img', output_img)
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False
    

cv2.namedWindow('insert_img', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('insert_img', process_img)
cv2.createTrackbar('size', 'insert_img', 10000, 50000, on_trackbar)

cv2.imshow('insert_img', output_img)
while True:
    # 等待键盘输入
    key = cv2.waitKey(1) & 0xFF
    if key == 13:  # Enter 键的 ASCII 值为 13
        cv2.imwrite('./output/final_output.png', output_img)  # 保存图像
        print("Image saved!")
        break  # 退出循环
    elif key == ord('q'):  # 按下 'q' 键退出
        break

cv2.destroyAllWindows()

(222, 474, 3)
(1365, 2048, 4)
Image saved!


KeyboardInterrupt: 

[[183 335]
 [183 336]
 [183 337]
 ...
 [281 214]
 [282 209]
 [282 210]]
[[183 335]
 [183 336]
 [183 337]
 ...
 [281 214]
 [282 209]
 [282 210]]
[[183 335]
 [183 336]
 [183 337]
 ...
 [281 214]
 [282 209]
 [282 210]]
